In [1]:
import pandas as pd
import numpy as np
import glob
import json

### First preprocess the screen data and create relevant files

In [2]:
df = pd.read_csv('/dfs/user/yhr/AI_RA/research_assistant/datasets/ground_truth_Scharenberg22.csv')

In [ ]:
import h5py

with h5py.File('/d', "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    # get first object name/key; may or may NOT be a group
    a_group_key = list(f.keys())[0]

    # get the object type for a_group_key: usually group or dataset
    print(type(f[a_group_key])) 

    # If a_group_key is a group name, 
    # this gets the object names in the group and returns as a list
    data = list(f[a_group_key])

    # If a_group_key is a dataset name, 
    # this gets the dataset values and returns as a list
    data = list(f[a_group_key])
    # preferred methods to get dataset values:
    ds_obj = f[a_group_key]      # returns as a h5py dataset object
    ds_arr = f[a_group_key][()]  #

In [58]:
sanchez.sort_values('Score')

,Gene,Score
10560,NSD1,-9.000665
5419,FBXW7,-7.915800
6021,GGNBP2,-7.661333
4658,EIF4G2,-7.271685
13472,RPRD1B,-6.793143
...,...,...
8567,LMOD2,-0.005625
1464,BCL7A,-0.003223
5466,FERD3L,-0.002953
17707,ZBTB14,-0.001429


In [61]:
screen_name = 'scharenberg_2022'
h5_file = os.path.join(save_directory, f"{screen_name}.h5")
if not os.path.exists(h5_file):
    df = pd.read_csv(f'/dfs/user/yhr/AI_RA/research_assistant/datasets/ground_truth_{screen_name}.csv')
    df = df.rename(columns={'Gene':'Gene_name'})

    df = df.set_index('Gene_name')
    
    gene_names = df.index.values.tolist()
    name_converter = HGNCNames(save_directory)
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    gene_names, data = df.index.values.tolist(), df[['Score']].values.astype(np.float32)


    HDF5Tools.save_h5_file(h5_file,
                           data,
                           f"scharenberg_2022_{screen_name}.h5",
                           column_names=["Score"],
                           row_names=gene_names)
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

,Symbol,RSA_Down,RSA_Up,Q1,Q3
10560,NSD1,-9.000665,-0.004267,-5.515584,-4.264298
5419,FBXW7,-7.915800,-0.004947,-5.451714,-2.803479
6021,GGNBP2,-7.661333,-0.032017,-3.298837,-2.773273
4658,EIF4G2,-7.271685,-0.001289,-4.175354,-2.592308
13472,RPRD1B,-6.793143,-0.045990,-2.973490,-2.250259
...,...,...,...,...,...
8567,LMOD2,-0.005625,-3.369090,1.459140,2.380598
1464,BCL7A,-0.003223,-4.064921,1.819689,3.312645
5466,FERD3L,-0.002953,-4.032406,1.801935,3.753448
17707,ZBTB14,-0.001429,-5.415197,1.716469,3.611752


In [55]:
sanchez_orig = pd.read_csv('/dfs/user/yhr/genedisco/genedisco/datasets/screens/sanchezetal2021_21days.csv')

In [54]:
sanchez = pd.read_csv('/dfs/user/yhr/AI_RA/research_assistant/datasets/ground_truth_Sanchez21.csv', index_col=0)

### Save topmovers

In [41]:
lower_thresh = np.percentile(sanchez['Score'].values,2.5)
upper_thresh = np.percentile(sanchez['Score'].values,97.5)

In [42]:
topmovers_sanchez = []

topmovers_sanchez.extend(sanchez[sanchez['Score']<lower_thresh]['Gene'].values)
topmovers_sanchez.extend(sanchez[sanchez['Score']>upper_thresh]['Gene'].values)

In [45]:
np.save('/dfs/user/yhr/AI_RA/research_assistant/datasets/topmovers_Sanchez21.npy', topmovers_sanchez)

### Write task prompts

In [48]:
task_prompt = {"Task": '', "Measurement": ''}
task_prompt["Task"] = f"identify genes that, when knocked out, either increase or decrease expression of endogenous tau protein levels in neurons, using an SH-SY5Y neuroblastoma cell line"
task_prompt["Measurement"] = f"the change in tau protein level compared to the non-targeting control, using a total tau antibody"    

file_path = f'./datasets/task_prompts/Sanchez21'+'.json'
with open(file_path, 'w') as file:
    json.dump(task_prompt, file)

In [49]:
task_prompt

{'Task': 'identify genes that, when knocked out, either increase or decrease expression of endogenous tau protein levels in neurons, using an SH-SY5Y neuroblastoma cell line',
 'Measurement': 'the change in tau protein level compared to the non-targeting control, using a total tau antibody'}